In [1]:
import numpy as np
from numpy.linalg import inv, multi_dot
from sympy import *
init_printing(use_unicode=True)

In [29]:
A = np.array([
    [2, 1, 1, 1, 0, 0],
    [4, 2, 3, 0, 1, 0],
    [2, 5, 5, 0, 0, 1]
], dtype=np.float64)
z = [[-1, -2, 1, 0, 0, 0]]
b = [0, 14, 28, 30]

A = np.r_[z, A]
A = np.c_[A, b]
A

array([[ -1.,  -2.,   1.,   0.,   0.,   0.,   0.],
       [  2.,   1.,   1.,   1.,   0.,   0.,  14.],
       [  4.,   2.,   3.,   0.,   1.,   0.,  28.],
       [  2.,   5.,   5.,   0.,   0.,   1.,  30.]])

**$x_2$ increases the most. $x_6$ is the blockage** 

In [30]:
A[3,:] = A[3,:] / 5.0
A

array([[ -1. ,  -2. ,   1. ,   0. ,   0. ,   0. ,   0. ],
       [  2. ,   1. ,   1. ,   1. ,   0. ,   0. ,  14. ],
       [  4. ,   2. ,   3. ,   0. ,   1. ,   0. ,  28. ],
       [  0.4,   1. ,   1. ,   0. ,   0. ,   0.2,   6. ]])

In [31]:
A[0,:] = A[0,:] + A[3,:] * 2
A[1,:] = A[1,:] + A[3,:] * -1  
A[2,:] = A[2,:] + A[3,:] * -2

A

array([[ -0.2,   0. ,   3. ,   0. ,   0. ,   0.4,  12. ],
       [  1.6,   0. ,   0. ,   1. ,   0. ,  -0.2,   8. ],
       [  3.2,   0. ,   1. ,   0. ,   1. ,  -0.4,  16. ],
       [  0.4,   1. ,   1. ,   0. ,   0. ,   0.2,   6. ]])

**$x_1$ increases the most. $x_4$ is the blockage**

In [35]:
A[1:,-1] / A[1:,0] 

array([  5.,   5.,  15.])

In [38]:
A[1,:] = A[1,:] * 1/A[1,0]
A

array([[ -0.2  ,   0.   ,   3.   ,   0.   ,   0.   ,   0.4  ,  12.   ],
       [  1.   ,   0.   ,   0.   ,   0.625,   0.   ,  -0.125,   5.   ],
       [  3.2  ,   0.   ,   1.   ,   0.   ,   1.   ,  -0.4  ,  16.   ],
       [  0.4  ,   1.   ,   1.   ,   0.   ,   0.   ,   0.2  ,   6.   ]])

In [59]:
A[0,:] = A[0,:] + A[1,:] * - A[0,0]
A[2,:] = A[2,:] + A[1,:] * - A[2,0]
A[3,:] = A[3,:] + A[1,:] * - A[3,0]
A

array([[  0.   ,   0.   ,   3.   ,   0.125,   0.   ,   0.375,  13.   ],
       [  1.   ,   0.   ,   0.   ,   0.625,   0.   ,  -0.125,   5.   ],
       [  0.   ,   0.   ,   1.   ,  -2.   ,   1.   ,   0.   ,   0.   ],
       [  0.   ,   1.   ,   1.   ,  -0.25 ,   0.   ,   0.25 ,   4.   ]])

# Dictionary 

In [24]:
x1, x2, x3, x4, x5, x6 = symbols("x1 x2 x3 x4 x5 x6")

In [4]:
def print_problem(Z, constraints):
    print("Z = ", Z)
    for c in constraints:
        print(c, "= 0")

In [25]:
# Initial
Z = x1 + 2*x2 - x3

c1 = 2*x1 +   x2 +   x3 + x4         - 14
c2 = 4*x1 + 2*x2 + 3*x3 +     x5     - 28
c3 = 2*x1 + 5*x2 + 5*x3 +         x6 - 30

print_problem(Z, [c1, c2, c3])

Z =  x1 + 2*x2 - x3
2*x1 + x2 + x3 + x4 - 14 = 0
4*x1 + 2*x2 + 3*x3 + x5 - 28 = 0
2*x1 + 5*x2 + 5*x3 + x6 - 30 = 0


In [26]:
# Finding where the blockage happens
print(solve(c1.subs({x1:0, x3:0, x4:0}), x2)) # In terms of x4
print(solve(c2.subs({x1:0, x3:0, x5:0}), x2)) # In terms of x5
print(solve(c3.subs({x1:0, x3:0, x6:0}), x2), "x6 goes out") # In terms of x6

[14]
[14]
[6] x6 goes out


In [27]:
#### Update the table ####
# third constraint in terms of x2
c3 = solve(c3, x2)[0]
c1 = c1.subs(x2, c3)
c2 = c2.subs(x2, c3)

Z = Z.subs(x2, c3)

print_problem(Z, [c1, c2, c3])

Z =  x1/5 - 3*x3 - 2*x6/5 + 12
8*x1/5 + x4 - x6/5 - 8 = 0
16*x1/5 + x3 + x5 - 2*x6/5 - 16 = 0
-2*x1/5 - x3 - x6/5 + 6 = 0


In [28]:
# Find new blockage (x1 increases the most)
print(solve(c1.subs({x3:0, x6:0, x4:0}), x1))
print(solve(c2.subs({x3:0, x6:0, x5:0}), x1))
print(solve(c3.subs({x3:0, x6:0, x2:0}), x1))

# Choose x4 to go out. It could've been either x4 or x5

[5]
[5]
[15]


In [31]:
c1 = solve(c1, x1)[0]
c2 = c2.subs(x1, c1)
c3 = c3.subs(x1, c1)

Z = Z.subs(x1, c1)

print_problem(Z, [c1, c2, c3])

Z =  -3*x3 - x4/8 - 3*x6/8 + 13
-5*x4/8 + x6/8 + 5 = 0
x3 - 2*x4 + x5 = 0
-x3 + x4/4 - x6/4 + 4 = 0


Since $\frac{\partial Z}{\partial x_i} \leq 0 \ \forall \ i$, we know we have reached the Max.

# Iterative Form

In [2]:
S = np.array([
    [-1, 2, 1, 0, 0, 0,  0],
    [ 2, 1, 1, 1, 0, 0, 14],
    [ 4, 2, 3, 0, 1, 0, 28],
    [ 2, 5, 5, 0, 0, 1, 30]
], dtype = np.float64)


In [3]:
in_v = 0
out_v = 1

S[out_v] = S[out_v] / S[out_v, in_v]
for ix, row in enumerate(S):
    if ix != out_v:
        S[ix] = row -  S[ix, in_v] * S[out_v]

In [4]:
S

array([[  0. ,   2.5,   1.5,   0.5,   0. ,   0. ,   7. ],
       [  1. ,   0.5,   0.5,   0.5,   0. ,   0. ,   7. ],
       [  0. ,   0. ,   1. ,  -2. ,   1. ,   0. ,   0. ],
       [  0. ,   4. ,   4. ,  -1. ,   0. ,   1. ,  16. ]])

# Matrix Form

In [47]:
def compute_parameters(A, C, b, svars):
    N = A[:, svars["nb"]]
    B = A[:, svars["b"]]
    C_b = C[:,svars["b"]]
    
    current_Zeq = multi_dot([C_b, inv(B), A]) - C
    solving_coefficients = inv(B).dot(b).T
    current_optimum = multi_dot([C_b, inv(B), b])
    
    return current_Zeq, solving_coefficients, current_optimum, inv(B).dot(A)

### Ex.1

In [48]:
A = np.array([
    [2, 1, 1, 1, 0, 0],
    [4, 2, 3, 0, 1, 0],
    [2, 5, 5, 0, 0, 1]
], dtype=np.float32)

C = np.array([[1, 2, -1, 0, 0, 0]])

b = np.array([[14, 28, 30]]).T

In [49]:
svars = {"b":  [3, 4, 1],
         "nb": [0, 5, 2]}

for v in iterate_simplex(A, C, b, svars):
    print(np.round((v.flatten()), 4), "\n")

[-0.2  0.   3.   0.   0.   0.4] 

[  8.  16.   6.] 

[ 12.] 



In [32]:
svars = {"b":  [0, 4, 1],
         "nb": [3, 5, 2]}

for v in iterate_simplex(A, C, b, svars):
    print(np.round((v.flatten()), 4), "\n")

[ 0.     0.     3.     0.125  0.     0.375] 

[ 5.  0.  4.] 

[ 13.] 



### Ex.2

In [61]:
A1 = np.array([
    [3, 7, 3, 1, 0],
    [2, 2, 6, 0, 1]
])

C1 = np.array([[60, 84, 72, 0, 0]])

b1 = np.array([[10, 4]]).T

In [62]:
svars1 = {"b":  [3, 4],
          "nb": [0, 1, 2]}


for v in compute_parameters(A1, C1, b1, svars1):
    print(np.round(v, 4), "\n\n")

[[-60. -84. -72.   0.   0.]] 


[[ 10.   4.]] 


[[ 0.]] 


[[ 3.  7.  3.  1.  0.]
 [ 2.  2.  6.  0.  1.]] 




In [63]:
svars1 = {"b":  [1, 4],
          "nb": [0, 3, 2]}


for v in compute_parameters(A1, C1, b1, svars1):
    print(np.round(v, 4), "\n\n") 

[[-24.   0. -36.  12.   0.]] 


[[ 1.4286  1.1429]] 


[[ 120.]] 


[[ 0.4286  1.      0.4286  0.1429  0.    ]
 [ 1.1429  0.      5.1429 -0.2857  1.    ]] 




In [64]:
svars1 = {"b":  [1, 2],
          "nb": [0, 3, 4]}


for v in compute_parameters(A1, C1, b1, svars1):
    print(np.round(v, 4), "\n\n")

[[-16.   0.   0.  10.   7.]] 


[[ 1.3333  0.2222]] 


[[ 128.]] 


[[ 0.3333  1.      0.      0.1667 -0.0833]
 [ 0.2222  0.      1.     -0.0556  0.1944]] 




In [65]:
svars1 = {"b":  [1, 0],
          "nb": [2, 3, 4]}


for v in compute_parameters(A1, C1, b1, svars1):
    print(np.round(v, 4), "\n\n")

[[  0.   0.  72.   6.  21.]] 


[[ 1.  1.]] 


[[ 144.]] 


[[ 0.     1.    -1.5    0.25  -0.375]
 [ 1.     0.     4.5   -0.25   0.875]] 


